# Visualization of "sample_scenario_00"
This is an example of data loading, basic manipulations and visualization of one of the scenarios.

First, we import the libraries that will be used in the notebook (you'll probably have to install them with pip, poetry or similar tools).

In [1]:
import pandas as pd
import geopandas as gpd
import folium
from pyproj import Geod

From "sample_predictions_empty.csv" we can obtain the target ICAO code and the arrival runway of the scenario.

In [2]:
TARGET_ICAO = "347247"
RUNWAY = "18R/36L"

Now we are going to load the geometries (geojson files) using geopandas (an extension of pandas that allow spatial querying and manipulations).

In [3]:
# Load the geometries
runways = gpd.read_file('runways.geojson')
thresholds = gpd.read_file('thresholds.geojson')

Now we can open the parquet file with pandas to load the dataframe:

In [4]:
# Load the scenario
scenario = pd.read_parquet('samples/sample_scenario_00.parquet')
scenario

,df,squawk,bds60_bds,bds60_heading,bds60_IAS,bds60_Mach,bds60_vrate_barometric,bds60_vrate_inertial,icao24,ts,...,heading,TAS,bds30_bds,bds30_issued_ra,bds30_terminated,bds30_multiple,year,month,day,hour
473055,20,<NA>,60,232.207031,263.0,0.776,-288.0,0.0,344699,1737545255855,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
473056,21,4516,<NA>,NaN,NaN,NaN,NaN,NaN,407fc1,1737545255855,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
473057,21,2203,<NA>,NaN,NaN,NaN,NaN,NaN,3964f1,1737545255855,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
473058,20,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,49514b,1737545255855,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
473059,21,6012,<NA>,NaN,NaN,NaN,NaN,NaN,344345,1737545255855,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679258,17,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,343141,1737545375089,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
679259,17,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,3991e2,1737545375089,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
679260,17,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,346384,1737545375089,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11
679261,17,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,346555,1737545375089,...,NaN,NaN,<NA>,<NA>,<NA>,<NA>,2025,1,22,11


This scenario contains data from multiple aircrafts. We are going to filter only the target aircraft for visualization, taking the columns that we want to plot and choosing the last position in the dataset.

In [5]:
# Take only the target aircraft
target_aircraft = scenario[scenario['icao24'] == TARGET_ICAO]
# Select only columns needed for visualization
target_aircraft = target_aircraft[["ts", "icao24", "lat_deg", "lon_deg", "groundspeed", "track"]]
# Forward fill the target aircraft data
target_aircraft = target_aircraft.ffill()
# Take only the last timestamp of the target aircraft for visualization
target_aircraft = target_aircraft[target_aircraft["ts"] == target_aircraft["ts"].max()]

The next block of code computes the lead vector that will be represented by extrapolating the position

In [6]:
geod = Geod(ellps='WGS84')

def extrapolate_position(
        ground_speed_knots:float, heading_deg:float,
        latitude_deg:float, longitude_deg:float,
        time_seconds: float
) -> tuple[float, float]:
    """
    Project the current position of the target aircraft to a future position based on the ground speed and time.
    """
    # Convert ground speed from knots to meters per second
    ground_speed_mps = ground_speed_knots * 0.514444
    # Calculate the distance in meters
    distance_m = ground_speed_mps * time_seconds
    # Calculate the lead vector
    lon_projected, lat_projected, _ = geod.fwd(lons=longitude_deg, lats=latitude_deg, az=heading_deg, dist=distance_m)
    return lat_projected, lon_projected

lead_vector_lat, lead_vector_lon = extrapolate_position(
    target_aircraft["groundspeed"].values[0], target_aircraft["track"].values[0],
    target_aircraft["lat_deg"].values[0], target_aircraft["lon_deg"].values[0],
    20)

We use the information from the csv file to select the runway and threshold point to be represented

In [7]:
runways = runways[runways["runway"] == RUNWAY]
thresholds = thresholds[thresholds["runway"] == RUNWAY]

Now we can plot everything.

In [8]:
madrid_map = folium.Map(location=[target_aircraft["lat_deg"].values[0], target_aircraft["lon_deg"].values[0]], zoom_start=9)

# Create popups for the geometries
popup_runways = folium.GeoJsonPopup(
    fields=["runway"],
    aliases=["Runway name"],
)

popup_thresholds = folium.GeoJsonPopup(
    fields=["runway"],
    aliases=["Runway name"],
)


# Add geometries
folium.GeoJson(runways, popup=popup_runways).add_to(madrid_map)
folium.GeoJson(thresholds, popup=popup_thresholds).add_to(madrid_map)
# Add target aircraft
folium.CircleMarker(
    [target_aircraft["lat_deg"].values[0], target_aircraft["lon_deg"].values[0]],
    radius=5, color='red', popup=f"ICAO: {target_aircraft['icao24'].values[0]}"
).add_to(madrid_map)
folium.PolyLine(
    [[target_aircraft["lat_deg"].values[0], target_aircraft["lon_deg"].values[0]], [lead_vector_lat, lead_vector_lon]],
    color='red', popup=f"ICAO: {target_aircraft['icao24'].values[0]}"
).add_to(madrid_map)
madrid_map